# Preprocessing to one SHS100k2 parquet file

In [2]:
import pandas as pd

shs100k2_test = pd.read_csv("../data/shs100k2_test.csv", sep=";")
shs100k2_train = pd.read_csv("../data/shs100k2_train.csv", sep=";")
shs100k2_val = pd.read_csv("../data/shs100k2_val.csv", sep=";")
shs_yt = pd.read_csv("../data/SHS-YT.csv", sep=";")
datacos = pd.read_csv("../data/da-tacos.csv", sep=";")
shs100k2_train["split"] = "TRAIN"
shs100k2_val["split"] = "VAL"
shs100k2_test["split"] = "TEST"
shs100k2 = pd.concat([shs100k2_test, shs100k2_val, shs100k2_train], axis=0, 
          ignore_index=True)

yt_metadata = pd.read_parquet("../data/yt_metadata.parquet")
yt_metadata = yt_metadata.rename({"title": "video_title"}, axis=1)


shsyt_yt = pd.merge(shs_yt, yt_metadata.reset_index(), how="left", on="yt_id").dropna(
    subset=["video_title"]
).drop(["link"], axis=1)
shsyt_yt.to_parquet("/home/hachmeier/data/shs_yt_yt.parquet")
datacos_yt = pd.merge(datacos, yt_metadata.reset_index(), how="left", on="yt_id").dropna(
    subset=["video_title"]
).drop(["link"], axis=1)
datacos_yt.to_parquet("/home/hachmeier/data/datacos_yt.parquet")
shs100k2_yt = pd.merge(shs100k2, yt_metadata.reset_index(), how="left", on="yt_id").dropna(
    subset=["video_title"]
).drop(["link"], axis=1)
shs100k2_yt.to_parquet("/home/hachmeier/data/shs100k2_yt.parquet")


In [2]:
import pandas as pd 

shs100k2_test = pd.read_parquet("/home/hachmeier/data/shs100k2_yt.parquet").query("split == 'TEST'")
shsyt = pd.read_parquet("/home/hachmeier/data/shs_yt_yt.parquet")
datacos = pd.read_parquet("/home/hachmeier/data/datacos_yt.parquet")


print("Crossjoin shsyt")
shsyt_pairs = pd.merge(shsyt, shsyt, how="cross", suffixes=["_a", "_b"])


Crossjoin shsyt


In [1]:
import pandas as pd

data = pd.read_parquet("/data/csi_datasets/shs100k2_yt.parquet")
data_L = data.sample(n=30_000)
data_M = data.sample(n=15_000)
data_S = data.sample(n=5_000)
data_XS = data.sample(n=1_000)


In [2]:
import pandas as pd

p="/home/repos/contrastive-product-matching/data/processed/shs100k_1000/contrastive/shs100k_1000-train.pkl.gz"
pd.read_json(p, lines=True)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [7]:
sample = data[["set_id", "yt_id"]].sample(10_000)
pd.merge(sample,sample,how="cross")



,set_id_x,yt_id_x,set_id_y,yt_id_y
0,3440,ox14nkmoV6k,3440,ox14nkmoV6k
1,3440,ox14nkmoV6k,473,mm0Ip_pIKT0
2,3440,ox14nkmoV6k,749,UchDcKJwHW8
3,3440,ox14nkmoV6k,2920,O2WpoKjAVzk
4,3440,ox14nkmoV6k,1454,pBM3x9x7VH8
...,...,...,...,...
99999995,2340,mwvf-dl6TOE,1693,TJPvm9DXhhY
99999996,2340,mwvf-dl6TOE,168,pWdGcgbR_5c
99999997,2340,mwvf-dl6TOE,252,NuwPcfgXSH4
99999998,2340,mwvf-dl6TOE,2560,Vc7cjtIGf6A


In [19]:
data_L.to_parquet("/data/csi_datasets/shs100k2_L_yt.parquet")
data_M.to_parquet("/data/csi_datasets/shs100k2_M_yt.parquet")
data_S.to_parquet("/data/csi_datasets/shs100k2_S_yt.parquet")
data_XS.to_parquet("/data/csi_datasets/shs100k2_XS_yt.parquet")



In [32]:
def gen_pairs(data, n_pos=1_000, factor_neg=6):

        n_cross = n_pos * 2
        n_neg = n_pos * factor_neg
        
        sample = data[["yt_id", "set_id"]].sample(n_cross)
        cross = pd.merge(sample, sample, how="cross", suffixes=["_a", "_b"])

        pos_pairs = cross.query("(yt_id_a != yt_id_b) & (set_id_a == set_id_b)").sample(n_pos)
        neg_pairs = cross.query("(yt_id_a != yt_id_b) & (set_id_a != set_id_b)").sample(n_neg)
        
        pos_pairs["label"] = 1
        neg_pairs["label"] = 0
        
        dataset = pd.concat(
            [pos_pairs, neg_pairs], 
            ignore_index=True).sample(frac=1) #.drop(["set_id_a", "set_id_b"])
        
        return dataset.rename({"yt_id_a": "ltable_id", "yt_id_b": "rtable_id"}, axis=1)

sample = gen_pairs(data)

KeyError: 'cross'

In [21]:
# Example DataFrame
data = {'set_id': [1, 2, 1, 3, 2, 3, 4, 4, 5, 5],
        'other_column': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

df = pd.DataFrame(data)

# Set the desired number of rows to sample (n)
n = 4

valid_sets = df['set_id'].groupby(df['set_id']).filter(lambda x: len(x) >= 2)

# Sample n rows from the filtered groups
sampled_df = df[df['set_id'].isin(valid_sets)].sample(n)




In [30]:
data.groupby(data['set_id']).filter(lambda x: len(x) >= 2)
sample = data[data['set_id'].isin(valid_sets)].sample(1000)




ValueError: Cannot take a larger sample than population when 'replace=False'

In [23]:
df

,set_id,other_column
0,1,10
1,2,20
2,1,30
3,3,40
4,2,50
5,3,60
6,4,70
7,4,80
8,5,90
9,5,100


In [3]:
print("Crossjoin shs100k2_test")
shs100k2_test_pairs = pd.merge(shs100k2_test, shs100k2_test, how="cross", suffixes=["_a", "_b"])



Crossjoin shs100k2_test


In [4]:
shs100k2_test_pairs

,set_id_a,ver_id_a,yt_id_a,title_a,performer_a,has_file_a,has_cqt_ch_a,has_cqt_20_a,has_crema_a,split_a,...,allow_ratings_b,average_rating_b,keywords_b,is_live_content_b,publish_date_b,upload_date_b,is_family_safe_b,category_b,is_live_now_b,keywords_tokenized_b
0,21,1,gORyrU1xQpg,Yesterday,Marianne Faithfull,True,True,True,True,TEST,...,True,NaN,"[yesterday, yestarday, marianne, faithfull, fa...",False,2010-06-03,2010-06-03,True,Music,False,"[paul, peac, 1967, new, faithful, faith, beatl..."
1,21,1,gORyrU1xQpg,Yesterday,Marianne Faithfull,True,True,True,True,TEST,...,True,NaN,"[Matt Monro (Musical Artist), Yesterday (Compo...",False,2013-11-15,2013-11-15,True,Entertainment,False,"[music, artist, composit, monro, yesterday, matt]"
2,21,1,gORyrU1xQpg,Yesterday,Marianne Faithfull,True,True,True,True,TEST,...,True,NaN,"[beatles, strings, instrumental, classical]",False,2010-08-13,2010-08-13,True,Music,False,"[classic, beatl, string, instrument]"
3,21,1,gORyrU1xQpg,Yesterday,Marianne Faithfull,True,True,True,True,TEST,...,True,NaN,"[yt:stretch=16:9, yt:crop=16:9, yt:quality=hig...",False,2013-01-10,2013-01-10,True,Music,False,"[ytcrop169, ytqualityhigh, cilla, black, yeste..."
4,21,1,gORyrU1xQpg,Yesterday,Marianne Faithfull,True,True,True,True,TEST,...,True,NaN,"[Sarah Vaughan, Marty Paich, The Beatles, Yest...",False,2010-09-15,2010-09-15,True,Music,False,"[paul, vaughan, sarah, marti, composit, mccart..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77633716,9998,2,-t8fkmDlRk8,I Don't Care,The Queers,True,True,True,True,TEST,...,True,NaN,"[53rd and 3rd, Shotgun Messiah, Ramones, Cover]",False,2010-03-15,2010-03-15,True,Music,False,"[messiah, shotgun, 3rd, cover, 53rd, ramon]"
77633717,9998,2,-t8fkmDlRk8,I Don't Care,The Queers,True,True,True,True,TEST,...,True,NaN,"[rock, punk, metallica, ramones, we're, happy,...",False,2008-08-23,2008-08-23,True,Music,False,"[ramon, rock, famili, 3rd, 53rd, metal, punk, ..."
77633718,9998,2,-t8fkmDlRk8,I Don't Care,The Queers,True,True,True,True,TEST,...,True,NaN,"[Otto Brandenburg (Musical Artist), Sytten Ti]",False,2015-04-17,2015-04-17,True,Music,False,"[music, ti, sytten, otto, brandenburg, artist]"
77633719,9998,2,-t8fkmDlRk8,I Don't Care,The Queers,True,True,True,True,TEST,...,True,NaN,"[rocket, to, russia, most popular ramones song...",False,2009-09-18,2009-09-18,True,Music,False,"[histori, garag, pop, view, elvi, album, russi..."


In [14]:
print("Crossjoin datacos")
datacos_pairs = pd.merge(datacos, datacos, how="cross", suffixes=["_a", "_b"])


Crossjoin datacos


: 

# Preprocess to abt-buy-like Format

### 1. Table file

In [32]:
shs100k2 = pd.read_parquet("/home/hachmeier/data/shs100k2_yt.parquet")

rel_cols = ["yt_id", "video_title", "channel_name", "description"]

# table
shs100k2[rel_cols].to_parquet(
    "/home/hachmeier/contrastive-product-matching/data/raw/shs100k2_yt/table.parquet"
                          )


,yt_id,video_title,channel_name,description
0,gORyrU1xQpg,Marianne Faithfull - Yesterday (with lyrics),FlierWithoutWings,"""Yesterday"" as beautifully sung by Marianne Fa..."
1,jQhC3bMMLmw,MATT MONRO - YESTERDAY,NANCYFLORESSANTOS,"""...all my troubles seemed\nso far away...""."
4,eI3-EVEU51s,Hollyridge Strings - Yesterday,boyjohn,Help support this channel by becoming a patron...
5,ew1y-gvO_NM,Cilla Black ::::: Yesterday.,SixtiesOnly,"""Yesterday"" is a song originally recorded by T..."
6,tjYQuEyxRp0,Sarah Vaughan - Yesterday,David Speed,"From 1981's ""Songs of the Beatles,"" (which sat..."
...,...,...,...,...
108515,2S3cauISpNw,The Sachal Ensemble - Give Me Love (Give Me Pe...,SachalEnsembleVEVO,Music video by The Sachal Ensemble performing ...
108516,Q6ix7lWPDAQ,Gary Stewart - Ain't Living Long Like This,steve fizzle,Gary Stewart - Ain't Living Long Like This
108518,0JP_ZipG5Z0,Rodney Crowell - I ain't living long like this,baalhabeit,"the original version,\n\nout of Rodney's Crowe..."
108520,j7MC3ckUaDA,Dream Syndicate - Ain't living Long Like This,Jay Dog,


### 2. Pair generation

In [79]:
def gen_pairs(data, n_pos=1500, n_neg=8500, n_for_cross=5000):

    sample = data[["yt_id", "set_id"]].sample(n_for_cross)
    cross = pd.merge(sample, sample, how="cross", suffixes=["_a", "_b"])

    pos_pairs = cross.query("(yt_id_a != yt_id_b) & (set_id_a == set_id_b)").sample(n_pos)
    neg_pairs = cross.query("(yt_id_a != yt_id_b) & (set_id_a != set_id_b)").sample(n_neg)
    
    pos_pairs["label"] = 1
    neg_pairs["label"] = 0
    
    dataset = pd.concat(
        [pos_pairs, neg_pairs], 
        ignore_index=True).sample(frac=1) #.drop(["set_id_a", "set_id_b"])
    
    return dataset

dataset = gen_pairs(shs100k2)



In [80]:
dataset

,yt_id_a,set_id_a,yt_id_b,set_id_b,label
4341,hfzOtbdrr10,1146,RJoO-GkeWDQ,940,0
2775,K-jvTipPL8A,3684,fbatSnN2Cn8,5067,0
941,ZC4qowOedfk,59,Mv9_OEgeG8Q,59,1
494,4gVClrDpyzw,214,kYnZsG7yq-U,214,1
9257,sfJNIeod4QU,70,PwfOdB8Dbgk,453,0
...,...,...,...,...,...
3144,7T3Shir-X50,8548,FxV_6Kp7PSw,412,0
2350,7K-WWlpawos,5327,5OAnWNWLM_Y,3320,0
8129,bl-8nuYSYzs,5033,3szo4Z2AZlw,2388,0
7248,Ha5lgrAUFrQ,49,j2WIUhibkVg,7507,0


,yt_id_a,set_id_a,yt_id_b,set_id_b
7822,MGxjIBEZvx0,2842,xcgCogM0_hg,2842
8696,MGxjIBEZvx0,2842,5oaG0jTC-R4,2842
11469,dCYzfkhE8_8,571,xFFPxvF00J8,571
11656,dCYzfkhE8_8,571,VdRvUn482Go,571
15400,QpW1pQ1-tMg,159,zKkrfsdGb2k,159
...,...,...,...,...
24984224,ZnekFd6_ABc,38,lIj8i54oMLA,38
24984939,ZnekFd6_ABc,38,satEWgJLGbQ,38
24992879,8WqwhEy1cK8,499,RP7QUY3cFjI,499
24993968,8WqwhEy1cK8,499,ol4O61bxzoA,499
